### 训练模型

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
#import pymysql
#pymysql.install_as_MySQLdb()
import MySQLdb as sql
import sklearn.mixture as mix

import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
#from pyecharts import Line

#不输出警告
import warnings
warnings.filterwarnings("ignore")
import time
import datetime
import pickle
from sklearn.externals import joblib

filesave=os.getcwd()+r'\online_test'

#0号门模型
a_open_sd_model_flie=r'\onlycan_opendata_sd_model0.pkl'
a_close_sd_model_flie=r'\onlycan_closedata_sd_model0.pkl'
a_open_normal_model_flie=r'\onlycan_opendata_normal_obj_model0.pkl'
a_close_normal_model_flie=r'\onlycan_closedata_normal_obj_model0.pkl'
#1号门模型
b_open_sd_model_flie=r'\onlycan_opendata_sd_model1.pkl'
b_close_sd_model_flie=r'\onlycan_closedata_sd_model1.pkl'
b_open_normal_model_flie=r'\onlycan_opendata_normal_obj_model1.pkl'
b_close_normal_model_flie=r'\onlycan_closedata_normal_obj_model1.pkl'

In [3]:
filesave+a_open_sd_model_flie

'\\online_test\\onlycan_opendata_sd_model0.pkl'

In [38]:
import threading
from sklearn import preprocessing

In [39]:
def count_gmm(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [40]:
#计算CV值
def calculate_cv(MG1,MG2):

    
    NumMixture1 = MG1.n_components
    NumMixture2 = MG2.n_components
    d = 0.0
    for k in range(0,NumMixture1):
        for j in range(0,NumMixture2):
            Sigma1 = MG1.covariances_[k]
            Mu1 = MG1.means_[k]
            Sigma2 = MG2.covariances_[j]
            Mu2 = MG2.means_[j]
            weight = MG1.weights_[k] * MG2.weights_[j]
            dist = L2Dist(Sigma1,Mu1,Sigma2,Mu2)
            d = d + weight*dist
    ''' the L2 norm of each GMM model'''
    n1 = 0.0
    n2 = 0.0
    for k in range(0,NumMixture1):
        for j in range(0,NumMixture1):
            Sigma1 = MG1.covariances_[k]
            Mu1 = MG1.means_[k]
            Sigma2 = MG1.covariances_[j]
            Mu2 = MG1.means_[j]
            weight = MG1.weights_[k]*MG1.weights_[j]
            n1 = n1 + weight * L2Dist(Sigma1,Mu1,Sigma2,Mu2)
    for k in range(0,NumMixture2):
        for j in range(0,NumMixture2):
            Sigma1 = MG2.covariances_[k]
            Mu1 = MG2.means_[k]
            Sigma2 = MG2.covariances_[j]
            Mu2 = MG2.means_[j]
            weight = MG2.weights_[k]*MG2.weights_[j]
            n2 = n2 + weight * L2Dist(Sigma1,Mu1,Sigma2,Mu2)
    cv = d/np.sqrt(n1)/np.sqrt(n2)
    return cv

In [41]:
#计算两个高斯分布之间的距离
def L2Dist(sigma1,mean1,sigma2,mean2):
    '''
    calculate the L2 distance between two multivariate Gaussian distributisons
    
    '''    
    n = mean1.size
    
    '''L2 norm of sigma1, mean1'''
    
    invsigma1 = np.linalg.inv(sigma1)
    invsigma2 = np.linalg.inv(sigma2)
    u = invsigma1*mean1 + invsigma2*mean2
    invsigma = np.linalg.inv(sigma1+sigma2)
    f1norm = 1/math.sqrt( math.sqrt( (2*math.pi)**n*np.linalg.det(sigma1)*2**n ) )
    '''L2 norm of sigma2, mean2'''
    f2norm = 1/math.sqrt( math.sqrt( (2*math.pi)**n*np.linalg.det(sigma2)*2**n ) )
    '''inner product of pdf f1 and f2'''
    
    InnerProd = 1/(math.sqrt(2*math.pi)**n) * math.sqrt(np.linalg.det(invsigma)) * \
                np.exp(-1/2*(-u.T*sigma1*invsigma*sigma2*u + mean1.T*invsigma1*mean1 + mean2.T*invsigma2*mean2))
    '''L2 distance between f1 and f2'''
    d = InnerProd/f1norm/f2norm
    
    return d

In [42]:
#训练高斯模型
def gmm_clustering(k,X,cov_type=None):
    '''
    基于高斯混合模型GMM进行无监督学习，聚类
    其中GMM采用传统的EM算法进行参数估计
    
    输入参数含义:
    k    ————   聚类的数目，必填项
    X    ————   样本的输入，比填项
    cov_type  ————  协方差类型，可选项
            四种选择：'spherical', 'tied', 'diag', 'full'
            如果不填写，则默认为'full'
        
    '''
    if cov_type is None:
        cov_type = 'full'   
    
    gmm = mix.GaussianMixture(n_components=k,covariance_type=cov_type)
    gmm.fit(X)
    
    return gmm

### 去除离群点

In [43]:
#去除异常数据规则：不满足均值 正负 3simga范围内的数据，均不保留
def filter_data(data):
    col =data.shape[1]
    for i in range(col):
        min_value = data.iloc[:,i].mean()-3*data.iloc[:,i].std()
        max_value = data.iloc[:,i].mean()+3*data.iloc[:,i].std()
        data.iloc[:,i][(data.iloc[:,i]>max_value) |(data.iloc[:,i]<min_value)] = np.nan
    data=data.dropna()#删除缺失数据
    return data
# filter_df=filter_data(df)
# filter_df=rawdata.ix[filter_df.index,:]
# filter_df=rawdata_can_open_filter.dropna()#删除缺失数据

### 开关门数据标准化函数

In [44]:
def open_StandardScaler(data_can_open,open_sd_model_flie):
    #去除离群点后的数据
    data_st1_sps = data_can_open.copy()
    #使用去除离群点的数据训练归一化模型
    #用去除故障点的数据进行训练
    standardscaler_st1 = preprocessing.StandardScaler()
    standardscaler_st1.fit(data_st1_sps)


    #用该模型将去除离群点和故障的数据进行标准化
    data_st1_sd_sps = standardscaler_st1.transform(data_st1_sps)
    data_st1_sd_sps = pd.DataFrame(data_st1_sd_sps)
    data_st1_sd_sps.index=data_st1_sps.index
    data_st1_sd_sps.columns=data_st1_sps.columns

###############################################################################    
    #保存开门数据标准化模型
    #保存训练好的模型文件
    joblib.dump(standardscaler_st1, filesave+open_sd_model_flie,compress=3)
    #导入训练好的模型文件
    #clf = joblib.load('onlycan_opendata_sd_model0.pkl') 
###############################################################################  
    return data_st1_sd_sps

In [23]:
def close_StandardScaler(data_can_close,close_sd_model_flie):
    #去除离群点后的数据
    data_st2_sps = data_can_close.copy()

    #使用去除离群点的数据训练归一化模型
    #用去除故障点的数据进行训练
    standardscaler_st2 = preprocessing.StandardScaler()
    standardscaler_st2.fit(data_st2_sps)

    #用该模型将去除离群点和故障的数据进行标准化
    data_st2_sd_sps = standardscaler_st2.transform(data_st2_sps)
    data_st2_sd_sps = pd.DataFrame(data_st2_sd_sps)
    data_st2_sd_sps.index=data_st2_sps.index
    data_st2_sd_sps.columns=data_st2_sps.columns
###############################################################################    
    #保存关门数据标准化模型
    #保存训练好的模型文件
    joblib.dump(standardscaler_st2, filesave+close_sd_model_flie,compress=3)
    #导入训练好的模型文件
#     clf = joblib.load('onlycan_closedata_sd_model0.pkl')
###############################################################################  
    return data_st2_sd_sps

### 开门数据寻优函数

In [24]:
def opendata_findbest(data_st1_sd_sps,open_normal_model_flie):
    time_start=time.time()
    #找出表现最好的行编号---开门状态
    #选取前50个点，计算cv均值
    list_findopenbest=[]
    sample_len=50
    print('open_sample_len',sample_len)
    i=0
    while len(list_findopenbest)!=sample_len:
        row_num=i
        i+=1
        k = 1
        cv1 = np.zeros(1)
        data_so = data_st1_sd_sps.copy()    
        data_s1 = data_so.iloc[row_num].copy()
        data_s1 = pd.DataFrame(data_s1)
        #print(data_so.index[row_num])

        data_s = data_st1_sd_sps.copy()

        normal_obj = gmm_clustering(k,data_s1)
        a = len(data_s)
        cv = np.zeros(a)

        #建立高斯混合模型
        def gmm_apply(x,k,normal_obj):
            data_l = pd.DataFrame(x)
            test_obj = gmm_clustering(k,data_l)
            y = calculate_cv(normal_obj,test_obj)#y为[[cv值]]，只返回cv值
            return y[0][0]

        #计算每一行的cv值，存在data_s的cv列中         
        data_s['cv']=data_s.apply(lambda row:gmm_apply(row,k,normal_obj), axis=1)

        #计算cv的均值
        cv1= data_s['cv']#cv1为Series
        #print(type(cv1))
        count =0    
        data_s['health_label']=cv1.map(count_gmm) 
        count=data_s['health_label'].sum()   
        c = cv1.mean() 
        ##########################################
        list_row=[data_so.index[row_num],a,count,c]
        list_findopenbest.append(list_row)

    df_findopenbest=pd.DataFrame(list_findopenbest,columns=['time','a', 'count', 'cv.mean'] )   
    #用cv.mean列对df进行排序
    df_openbest=df_findopenbest.sort_values(by="cv.mean" , ascending=False)#按照降序排序

    time_end=time.time()
    print('findopenbest totally cost',datetime.timedelta(seconds=time_end-time_start))
    print('df_openbest',df_openbest.head())
##########################################
    #保存效果最好的模型
    data_so = data_st1_sd_sps.copy()
    best_time_df=df_openbest.iloc[[0],[0]].values
    print(best_time_df)
    for i in best_time_df:
        for j in i:
            best_time=j        
    print(best_time)
    data_s1 = data_so.loc[best_time].copy()
    data_s1 = pd.DataFrame(data_s1)
    normal_obj = gmm_clustering(1,data_s1)
    #保存normal_obj模型
    #保存训练好的模型文件
    joblib.dump(normal_obj, filesave+open_normal_model_flie,compress=3)
    #导入训练好的模型文件
#     clf = joblib.load('onlycan_opendata_normal_obj_model0.pkl') 
    

### 关门数据寻优函数

In [25]:
def closedata_findbest(data_st2_sd_sps,close_normal_model_flie):
    time_start=time.time()
    #找出表现最好的行编号---关门状态
    #选取前50个点，计算cv均值
    list_findclosebest=[]
    sample_len=50
    print('close_sample_len',sample_len)
    i=0
    while len(list_findclosebest)!=sample_len:
        row_num=i
        i+=1
        k = 1
        cv1 = np.zeros(1)
        data_so = data_st2_sd_sps.copy()    
        data_s1 = data_so.iloc[row_num].copy()
        data_s1 = pd.DataFrame(data_s1)
        
        data_s = data_st2_sd_sps.copy()

        normal_obj = gmm_clustering(k,data_s1)
        a = len(data_s)
        cv = np.zeros(a)

        #建立高斯混合模型
        def gmm_apply(x,k,normal_obj):
            data_l = pd.DataFrame(x)
            test_obj = gmm_clustering(k,data_l)
            y = calculate_cv(normal_obj,test_obj)#y为[[cv值]]，只返回cv值
            return y[0][0]

        #计算每一行的cv值，存在data_s的cv列中         
        data_s['cv']=data_s.apply(lambda row:gmm_apply(row,k,normal_obj), axis=1)

        #计算cv的均值
        cv1= data_s['cv']#cv1为Series
        #print(type(cv1))
        count =0    
        data_s['health_label']=cv1.map(count_gmm) 
        count=data_s['health_label'].sum()   
        c = cv1.mean() 
        ##########################################
        list_row=[data_so.index[row_num],a,count,c]
        list_findclosebest.append(list_row)

    df_findclosebest=pd.DataFrame(list_findclosebest,columns=['time','a', 'count', 'cv.mean'] )   
    #用cv.mean列对df进行排序
    df_closebest=df_findclosebest.sort_values(by="cv.mean" , ascending=False)#按照降序排序

    time_end=time.time()
    print('findclosebest totally cost',datetime.timedelta(seconds=time_end-time_start))
    print('df_closebest',df_closebest.head())
##########################################
    #保存效果最好的模型
    data_so = data_st2_sd_sps.copy()
    best_time_df=df_closebest.iloc[[0],[0]].values
    for i in best_time_df:
        for j in i:
            best_time=j        
    print(best_time)
    data_s1 = data_so.loc[best_time].copy()
#     data_s1 = data_so.loc[df_closebest.ix[[0],['time']]].copy()
    data_s1 = pd.DataFrame(data_s1)
    normal_obj = gmm_clustering(1,data_s1)
    #保存normal_obj模型
    #保存训练好的模型文件
    joblib.dump(normal_obj, filesave+close_normal_model_flie,compress=3)
    #导入训练好的模型文件
#     clf = joblib.load('onlycan_closedata_normal_obj_model0.pkl') 

### 自动定位寻优区间

In [26]:
def findbestperiod(rawdata_can_open):
    #当doortime前两位一致的时候对em进行排序,万一一天运行出的数据不足500条怎么办
    time_start=time.time()
    period=0
    findperiod_list=[]
    
    DOOR_STATE_TIME_mode=rawdata_can_open['DOOR_STATE_TIME'].mode()[0]#众数
    print('DOOR_STATE_TIME_mode',DOOR_STATE_TIME_mode)
    oneday=datetime.timedelta(days=1) #1天，用于+1天操作
    can_date=rawdata_can_open.index[0].date()
    end_can_date=rawdata_can_open.index[-1].date()
    #print (end_can_date)
    #can_date=datetime.datetime.strptime(can_date_str, '%Y-%m-%d')#从str格式转回datetime
    
    while(1):
        #print (can_date)
        if can_date==end_can_date+oneday:#超过最后的日期
            break
        else:
            can_date_str=can_date.strftime('%Y-%m-%d')#转化为字符串
            oneday_data=rawdata_can_open.ix[can_date_str].copy()
            #print (len(oneday_data))
            if len(oneday_data)>100:#判断标准化需要的数据足够

                door_state_time_count=int(100*len(oneday_data[oneday_data['DOOR_STATE_TIME']==DOOR_STATE_TIME_mode])/len(oneday_data))#计算该天内DOOR_STATE_TIME值等于众数的百分比
                em_mean=abs(oneday_data['EM'].mean())
                list_row=[can_date_str,door_state_time_count,em_mean]
                findperiod_list.append(list_row)
                can_date=can_date+oneday#加一天  
            else:
                can_date=can_date+oneday#加一天


    df_findperiod=pd.DataFrame(findperiod_list,columns=['time','door_state_time_count', 'EM_mean'] )   
    #对door_state_time_count占比达标的period根据em进行升序排序
    df_findperiod=df_findperiod[df_findperiod['door_state_time_count']>95]
    df_findperiod=df_findperiod.sort_values(by=['EM_mean'] , ascending=[True])#按照降序排序

    time_end=time.time()
    print('findperiod totally cost',datetime.timedelta(seconds=time_end-time_start))
    print(df_findperiod)
    period_df=df_findperiod.iloc[[0],[0]].values
    for i in period_df:
        for j in i:
            period=j 
    print('most nomorl period is',period)
    return period

### 主函数

In [49]:
#需输入门编号gate 寻优时间段period

# #连接本地数据库
# config = {'user': 'root',
#             'passwd': 'ABCabc123',
#             'host': '127.0.0.1',
#             'db': 'maoqing'}
#连接外部数据库
config = {'user': 'root',
             'passwd': 'ABCabc123',
             'host': '10.84.1.20',
             'db': 'maoqing'}
database='door_data_can0812'

sql_statement="select * from "+database+" where HAPPENTIME > '2019-03-18' and GATE = 1"#and HAPPENTIME < '2019-04-10

#def start():

#链接数据库 拉一天的正常运行数据
conn = sql.connect(**config)
cur = conn.cursor(cursorclass=sql.cursors.DictCursor)#返回字段名
#根据read_flag为null筛选出新录入的数据
#cur.execute("select * from "+database+" where read_flag is null")#door_data_can 为原始数据表名
#data=cur.fetchall()#执行读取操作,data 为dict
#rawdata_can = pd.DataFrame(list(data))#将读取结果转为dataframe格式
rawdata_can=pd.read_sql(sql_statement,conn)
rawdata_can['HAPPENTIME']=pd.to_datetime(rawdata_can['HAPPENTIME'])
rawdata_can=rawdata_can.set_index('HAPPENTIME')
rawdata_can=rawdata_can.loc['2019/3/12':,['E_Q', 'E_D', 'SPEED_SUM',  'SPEED_MAX_P',
       'SPEED_MAX_N', 'SPEED_MAX_N_POSITION', 'START_POSITION', 'END_POSITION',
       'IQ_SUM_P', 'IQ_SUM_N', 'ID_SUM_P', 'ID_SUM_N', 'EM','SPEED_SUM_SQR','SPEED_SUM_CNT','DOOR_STATE_TIME','DOOR_STATE','GATE','DOOR_STATE_Z']].copy()
rawdata_can=rawdata_can.dropna()#删除有缺失值的行

rawdata_can.head()

,E_Q,E_D,SPEED_SUM,SPEED_MAX_P,SPEED_MAX_N,SPEED_MAX_N_POSITION,START_POSITION,END_POSITION,IQ_SUM_P,IQ_SUM_N,ID_SUM_P,ID_SUM_N,EM,SPEED_SUM_SQR,SPEED_SUM_CNT,DOOR_STATE_TIME,DOOR_STATE,GATE,DOOR_STATE_Z
HAPPENTIME,,,,,,,,,,,,,,,,,,,
2019-03-18 00:00:05,2,-1,460,60,-20,35745,35745,35745,62005,0,148,3192,-1,18000,39,39,0010001000000010,1,0010101000100010
2019-03-18 00:00:05,0,0,0,0,0,0,35745,35745,0,0,0,0,0,0,0,25315,0000001000100110,1,0010001000000010
2019-03-18 00:00:07,13,-1,-2104,0,-96,35719,35745,35587,241,493117,4794,5908,1,114448,45,45,0010011000100111,1,0000001000100110
2019-03-18 00:00:17,220,-1,-1362,100,-576,22015,35587,16582,0,5472586,62669,63121,144,1023118,419,419,0010011100100101,1,0010011000100111
2019-03-18 00:00:34,7415,-2,-954075,164,-961,16582,16582,16582,0,512081249,22841,66662,-2,526484555,1821,1821,0010010100100001,1,0010011100100101


In [50]:
#统计读取出来的数据中有多少个门的数据
rawdata_can_group=rawdata_can.groupby('GATE').count()
for door_gate in rawdata_can_group.index:
    print ('door_gate',door_gate)

door_gate 1


### 对0号门数据进行寻优

In [29]:
#筛选开门数据
# rawdata_can_open=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010011100100101')&(rawdata_can['GATE'].astype(str)==str(0))]  
rawdata_can_open1=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010011100100101')&(rawdata_can['GATE'].astype(str)==str(0))].loc[:'2019-04-01',:]
rawdata_can_open2=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010011100100101')&(rawdata_can['GATE'].astype(str)==str(0))&(rawdata_can['DOOR_STATE_Z'].astype(str)=='0010011000100111')].loc['2019-04-01':,:]
rawdata_can_open=rawdata_can_open1.append(rawdata_can_open2)
# rawdata_can_open=rawdata_can_open.loc['2019/3/12':,['E_Q', 'E_D', 'SPEED_SUM',  'SPEED_MAX_P',
#    'SPEED_MAX_N', 'SPEED_MAX_N_POSITION', 'START_POSITION', 'END_POSITION',
#    'IQ_SUM_P', 'IQ_SUM_N', 'ID_SUM_P', 'ID_SUM_N', 'EM','SPEED_SUM_SQR','SPEED_SUM_CNT','DOOR_STATE_TIME']].copy()
# rawdata_can_open=rawdata_can_open.dropna()#删除缺失数据
rawdata_can_open['SPEED_SUM_SQR_REAL'] = rawdata_can_open['SPEED_SUM_SQR']/rawdata_can_open['SPEED_SUM_CNT']
rawdata_can_open=rawdata_can_open.drop(labels=['SPEED_SUM_SQR','SPEED_SUM_CNT','GATE','DOOR_STATE','DOOR_STATE_Z'],axis=1)#删除计算后的两列
#print(rawdata_can_open.head())

#筛选关门数据
rawdata_can_close=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010101100101010')&(rawdata_can['GATE'].astype(str)==str(0))]  
# rawdata_can_close=rawdata_can_close.loc['2019/3/12':,['E_Q', 'E_D', 'SPEED_SUM',  'SPEED_MAX_P',
#    'SPEED_MAX_N', 'SPEED_MAX_N_POSITION', 'START_POSITION', 'END_POSITION',
#    'IQ_SUM_P', 'IQ_SUM_N', 'ID_SUM_P', 'ID_SUM_N', 'EM','SPEED_SUM_SQR','SPEED_SUM_CNT','DOOR_STATE_TIME']].copy()
# rawdata_can_close=rawdata_can_close.dropna()#删除缺失数据
rawdata_can_close['SPEED_SUM_SQR_REAL'] = rawdata_can_close['SPEED_SUM_SQR']/rawdata_can_close['SPEED_SUM_CNT']
rawdata_can_close=rawdata_can_close.drop(labels=['SPEED_SUM_SQR','SPEED_SUM_CNT','GATE','DOOR_STATE','DOOR_STATE_Z'],axis=1)#删除计算后的两列
# print(rawdata_can_close.head())
# rawdata_can_open.head()
#rawdata_can_close.head()
#进行正常周期筛选################################################################
open_period=findbestperiod(rawdata_can_open)
# open_period="2019-03-16" 
rawdata_can_open=rawdata_can_open.drop(labels=['DOOR_STATE_TIME'],axis=1)#删除'DOOR_STATE_TIME'列
rawdata_can_open=rawdata_can_open[open_period].copy()    
#print(rawdata_can_open.head())
#
close_period=findbestperiod(rawdata_can_close)
# close_period="2019-03-16" 
rawdata_can_close=rawdata_can_close.drop(labels=['DOOR_STATE_TIME'],axis=1)#删除'DOOR_STATE_TIME'列
rawdata_can_close=rawdata_can_close[close_period].copy()    
#print(rawdata_can_close.head())



DOOR_STATE_TIME_mode 377
findperiod totally cost 0:00:00.065855
          time  door_state_time_count     EM_mean
27  2019-04-08                     98  155.240612
32  2019-04-13                     98  155.293463
31  2019-04-12                     99  155.328929
38  2019-04-19                     98  155.338665
30  2019-04-11                     99  155.399861
34  2019-04-15                     99  155.466620
26  2019-04-07                     98  155.468011
28  2019-04-09                     99  155.474600
39  2019-04-20                     96  155.479833
29  2019-04-10                     99  155.502782
35  2019-04-16                     99  155.536857
41  2019-04-22                     97  155.557719
40  2019-04-21                     99  155.559110
33  2019-04-14                     99  155.578874
37  2019-04-18                     98  155.606398
36  2019-04-17                     99  155.706333
25  2019-04-06                     99  155.858136
24  2019-04-05                    10

In [30]:
###############################################################################    
#去除离群点数据
data_can_open=filter_data(rawdata_can_open)
data_can_close=filter_data(rawdata_can_close)
data_can_open.head()
###############################################################################
#去除离群点后的数据,对所有的特征做标准化化处理,并保存标准化模型
data_st1_sd_sps = open_StandardScaler(data_can_open,a_open_sd_model_flie)
data_st2_sd_sps = close_StandardScaler(data_can_close,a_close_sd_model_flie)
data_st1_sd_sps.head()

###############################################################################
#启动线程对开门和关门数据进行寻优并保存最优的模型
time_start=time.time()
threading.Thread(target=opendata_findbest(data_st1_sd_sps,a_open_normal_model_flie)).start()
threading.Thread(target=closedata_findbest(data_st2_sd_sps,a_close_normal_model_flie)).start()
time_end=time.time()
print('findbest totally cost',datetime.timedelta(seconds=time_end-time_start))

open_sample_len 50
findopenbest totally cost 0:01:58.748694
df_openbest                   time     a  count   cv.mean
6  2019-04-08 00:07:10  1376   1375  0.949678
26 2019-04-08 00:27:10  1376   1375  0.948461
11 2019-04-08 00:12:10  1376   1375  0.948129
39 2019-04-08 00:40:10  1376   1375  0.946094
3  2019-04-08 00:04:10  1376   1375  0.941176
[['2019-04-08T00:07:10.000000000']]
2019-04-08T00:07:10.000000000
close_sample_len 50
findclosebest totally cost 0:01:54.823884
df_closebest                   time     a  count   cv.mean
6  2019-03-12 00:07:43  1336   1336  0.945736
7  2019-03-12 00:08:43  1336   1336  0.944998
3  2019-03-12 00:04:43  1336   1336  0.942659
31 2019-03-12 00:33:42  1336   1336  0.941164
44 2019-03-12 00:46:42  1336   1336  0.941027
2019-03-12T00:07:43.000000000
findbest totally cost 0:03:53.585546


### 对1号门数据进行寻优

In [61]:
#筛选开门数据
rawdata_can_open=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010011100100101')&(rawdata_can['GATE'].astype(str)==str(1))]  
rawdata_can_open=rawdata_can_open.loc['2019/3/19':,:].copy()#限制寻优范围
# rawdata_can_open=rawdata_can_open.dropna()#删除缺失数据
rawdata_can_open['SPEED_SUM_SQR_REAL'] = rawdata_can_open['SPEED_SUM_SQR']/rawdata_can_open['SPEED_SUM_CNT']
rawdata_can_open=rawdata_can_open.drop(labels=['SPEED_SUM_SQR','SPEED_SUM_CNT','GATE','DOOR_STATE','DOOR_STATE_Z'],axis=1)#删除计算后的两列
#print(rawdata_can_open.head())

#筛选关门数据
rawdata_can_close=rawdata_can[(rawdata_can['DOOR_STATE'].astype(str)=='0010101100101010')&(rawdata_can['GATE'].astype(str)==str(1))]  
rawdata_can_close=rawdata_can_close.loc['2019/3/19':,:].copy()#限制寻优范围
# rawdata_can_close=rawdata_can_close.dropna()#删除缺失数据
rawdata_can_close['SPEED_SUM_SQR_REAL'] = rawdata_can_close['SPEED_SUM_SQR']/rawdata_can_close['SPEED_SUM_CNT']
rawdata_can_close=rawdata_can_close.drop(labels=['SPEED_SUM_SQR','SPEED_SUM_CNT','GATE','DOOR_STATE','DOOR_STATE_Z'],axis=1)#删除计算后的两列
#print(rawdata_can_close.head())
rawdata_can_open.head()
#rawdata_can_close.head()
#进行正常周期筛选################################################################
open_period=findbestperiod(rawdata_can_open)
close_period=findbestperiod(rawdata_can_close)


DOOR_STATE_TIME_mode 421
findperiod totally cost 0:00:00.166554
           time  door_state_time_count     EM_mean
37   2019-04-25                    100  195.954798
44   2019-05-02                    100  195.982615
117  2019-07-14                    100  196.425990
38   2019-04-26                    100  196.705146
118  2019-07-15                    100  196.888039
43   2019-05-01                    100  197.551460
116  2019-07-13                    100  197.594576
115  2019-07-12                    100  197.717663
127  2019-07-24                    100  197.753129
128  2019-07-25                    100  197.854760
59   2019-05-17                    100  198.010431
36   2019-04-24                    100  198.160640
120  2019-07-17                    100  198.399861
62   2019-05-20                    100  198.408901
132  2019-07-29                    100  198.601530
60   2019-05-18                    100  198.849096
102  2019-06-29                    100  198.901947
58   2019-05-16   

In [65]:
#截取正常运行 ################################################################
open_period="2019-05-01" 
rawdata_can_open_cut=rawdata_can_open.drop(labels=['DOOR_STATE_TIME'],axis=1)#删除'DOOR_STATE_TIME'列
rawdata_can_open_cut=rawdata_can_open_cut[open_period].copy()    
#print(rawdata_can_open.head())

close_period="2019-05-02" 
rawdata_can_close_cut=rawdata_can_close.drop(labels=['DOOR_STATE_TIME'],axis=1)#删除'DOOR_STATE_TIME'列
rawdata_can_close_cut=rawdata_can_close_cut[close_period].copy()    
#print(rawdata_can_close.head())

In [66]:
###############################################################################    
#去除离群点数据
data_can_open_cut=filter_data(rawdata_can_open_cut)
data_can_close_cut=filter_data(rawdata_can_close_cut)

###############################################################################
#去除离群点后的数据,对所有的特征做标准化化处理,并保存标准化模型
data_st1_sd_sps = open_StandardScaler(data_can_open_cut,b_open_sd_model_flie)
data_st2_sd_sps = close_StandardScaler(data_can_close_cut,b_close_sd_model_flie)
data_st1_sd_sps.head()



,E_Q,E_D,SPEED_SUM,SPEED_MAX_P,SPEED_MAX_N,SPEED_MAX_N_POSITION,START_POSITION,END_POSITION,IQ_SUM_P,IQ_SUM_N,ID_SUM_P,ID_SUM_N,EM,SPEED_SUM_SQR_REAL
HAPPENTIME,,,,,,,,,,,,,,
2019-05-01 00:00:04,-0.004016,0.0,-2.072182,2.257055,-2.359357,1.732430,1.441696,0.0,0.0,1.267363,0.725407,0.470552,1.364754,1.592845
2019-05-01 00:01:04,0.285552,0.0,-1.086970,-0.315141,0.297708,0.869992,1.441696,0.0,0.0,0.789679,-0.269839,-1.419021,1.364754,-1.265394
2019-05-01 00:02:04,-0.293584,0.0,-1.579576,-0.315141,0.297708,0.869992,-0.227893,0.0,0.0,0.847931,1.424225,1.456757,1.364754,-1.589458
2019-05-01 00:03:04,-0.872721,0.0,-2.072182,-1.086800,0.297708,1.732430,-0.227893,0.0,0.0,0.642077,0.550265,0.971078,0.272632,-1.398909
2019-05-01 00:05:04,-0.872721,0.0,-1.579576,-0.315141,-1.030825,0.007554,-1.897481,0.0,0.0,0.582627,-0.764178,0.420148,0.272632,0.668616


In [67]:
###############################################################################
#启动线程对开门和关门数据进行寻优并保存最优的模型
time_start=time.time()
# threading.Thread(target=opendata_findbest(data_st1_sd_sps,b_open_normal_model_flie)).start()#开门模型训练保存
threading.Thread(target=closedata_findbest(data_st2_sd_sps,b_close_normal_model_flie)).start()#关门模型训练保存
time_end=time.time()
print('findbest totally cost',datetime.timedelta(seconds=time_end-time_start))

close_sample_len 50
findclosebest totally cost 0:01:51.910910
df_closebest                   time     a  count   cv.mean
35 2019-05-02 00:49:32  1260   1260  0.929659
43 2019-05-02 00:57:32  1260   1260  0.928956
17 2019-05-02 00:26:32  1260   1260  0.928875
31 2019-05-02 00:45:32  1260   1260  0.928279
44 2019-05-02 00:58:32  1260   1260  0.927227
2019-05-02T00:49:32.000000000
findbest totally cost 0:01:51.916894


### 测试用，保存normal模型

In [14]:
#保存指定的开门模型
data_s1 = data_st1_sd_sps.loc['2019-04-08 00:46:45'].copy()
data_s1 = pd.DataFrame(data_s1)
normal_obj = gmm_clustering(1,data_s1)
#保存normal_obj模型
#保存训练好的模型文件
joblib.dump(normal_obj, 'onlycan_opendata_normal_obj_model1.pkl',compress=3)
#导入训练好的模型文件
clf = joblib.load('onlycan_opendata_normal_obj_model1.pkl')

NameError: name 'data_st1_sd_sps' is not defined